# This assignment consist of 3 parts i.e. Part - 1, Part - 2 & Part - 3. This notebook consist of Part -1 only.

## Part - 1

### Use BeautifulSoup website scraping library to transform the data in the table on this Wikipedia page into pandas dataframe

In [1]:
#import Beatifulsoup
import requests
from bs4 import BeautifulSoup

In [2]:
#create an object with the data from the Wikipedia page
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(website_url)

In [3]:
#find the table in the website
My_table = soup.find('table',{'class':'wikitable sortable'})

In [4]:
#extract the table data of Wikipedia
WP_data = [] 
for row in My_table.find_all("tr"):
    cols = row.find_all("td")
    cols = [ele.text.strip() for ele in cols]
    WP_data.append(cols)

### Create a dataframe

In [9]:
#create a panda dataframe
import pandas as pd
df = pd.DataFrame(WP_data)

#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = df.rename(columns={0:"Postcode",1:"Borough",2:"Neighborhood"})
df.head()

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [12]:
#Only process the cells that have an assigned borough:
#1)drop the first row
df2 = df.copy()
df2 = df.drop([0])
#2)drop cells with a borough that is Not assigned
df2 = df2.drop(df2[df2['Borough']=='Not assigned'].index)

df2 = df2.reset_index(drop=True)
df2.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [13]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df3 = df2.copy()
df3['Neighborhood'] = df3.apply(
    lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'],
    axis=1
)

In [14]:

#if there is more than one neighborhood in one postal code area, combine those rows and separate the neighborhoods with a comma (group the dataframe by postcode and borough,and then join the neighborhoods)
df4 = df3.copy()
df4 = (df4.groupby(['Postcode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

df4 = pd.DataFrame(df4)

df4.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df4.shape

(103, 3)